<a href="https://colab.research.google.com/github/ianzin30/CNN-RedesNeurais/blob/main/CNN_RedesNeurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports e bibliotecas

In [ ]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten
from keras import backend as k


# Carregando e Preprocessando os Dados


In [ ]:
# Carregamento dos dados
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Definição das dimensões da imagem
img_rows, img_cols = 28, 28

In [ ]:
# Carregamento dos dados
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Definição das dimensões da imagem
img_rows, img_cols = 28, 28

# Reformatando as imagens com base na configuração de canal
#def format_data(x_train, x_test, img_rows, img_cols):
 #   if keras.backend.image_data_format() == 'channels_first':
   #     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    #    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
     #   inpx = (1, img_rows, img_cols)
    #else:
     #   x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
      #  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
       # inpx = (img_rows, img_cols, 1)

    #return x_train, x_test, inpx

#format_data(x_train,x_test,img_rows,img_cols)

# Reformatando as imagens com base na configuração de canal
if k.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    inpx = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    inpx = (img_rows, img_cols, 1)

# Convertendo os dados para float e normalizando
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


# Transformando os rótulos ( labels ) em categorias


In [ ]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)


# Rede Convolucional Simples

Primeiro teste simples sobre redes convolucionais. A arquitetura da rede inclui uma camada convolucional, uma de flattening e densas.

## Arquitetura da Rede


1. **Primeira Camada Convolucional**: Aplicamos 32 filtros de tamanho 3x3 com função de ativação ReLU.
2. **Flatten**: Convertendo a saída da camada convolucional em um vetor 1D.
3. **Primeira Camada Densa**: Contém 250 neurônios com função de ativação sigmoide.
4. **Camada de Saída**: Contém 10 neurônios (um para cada classe) com função de ativação softmax.

# mudança

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()


In [ ]:
inpx2 = Input(shape=(inpx))
layer1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(inpx2)
flattened = Flatten()(layer1)  # Add this line to flatten the output of the Conv2D layer
layer2 = Dense(250, activation='sigmoid')(flattened)
layer3 = Dense(10, activation='softmax')(layer2)



In [ ]:
model = Model([inpx2], layer3)
model.compile(optimizer=keras.optimizers.Adadelta(),
			loss=keras.losses.categorical_crossentropy,
			metrics=['accuracy'])

model.fit(x_train, y_train, epochs=240, batch_size=500)

Epoch 1/2
120/120 [==============================] - 2s 10ms/step - loss: 2.5797 - accuracy: 0.0904
Epoch 2/2
120/120 [==============================] - 1s 9ms/step - loss: 2.5402 - accuracy: 0.0903


In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])


loss= 0.31196504831314087
accuracy= 0.9154000282287598


In [ ]:
def treinamento(primeiro_layer,epocas,ultimo_layer):
  model = Model([inpx], ultimo_layer)
  model.compile(optimizer=keras.optimizers.Adadelta(),
        loss=keras.losses.categorical_crossentropy,
        metrics=['accuracy'])

  model.fit(x_train, y_train, epochs=epocas, batch_size=500)

In [ ]:
def stats(x):
  score = x.evaluate(x_test, y_test, verbose=0)
  print('loss=', score[0])
  print('accuracy=', score[1])

# Testes com diferentes tamanhos de filtro

### Teste 1 (5x5)

In [ ]:
teste1 = Input(shape=(inpx))
layer1 = Conv2D(32, kernel_size=(5, 5), activation='relu')(teste1)
flattened = Flatten()(layer1)  # Add this line to flatten the output of the Conv2D layer
layer2 = Dense(250, activation='sigmoid')(flattened)
layer3 = Dense(10, activation='softmax')(layer2)

In [ ]:
model = Model([teste1], layer3)
model.compile(optimizer=keras.optimizers.Adadelta(),
			loss=keras.losses.categorical_crossentropy,
			metrics=['accuracy'])

model.fit(x_train, y_train, epochs=240, batch_size=500)


Epoch 1/240
120/120 [==============================] - 3s 12ms/step - loss: 2.4336 - accuracy: 0.0975
Epoch 2/240
120/120 [==============================] - 1s 9ms/step - loss: 2.4048 - accuracy: 0.0976
Epoch 3/240
120/120 [==============================] - 1s 9ms/step - loss: 2.3769 - accuracy: 0.0988
Epoch 4/240
120/120 [==============================] - 1s 9ms/step - loss: 2.3494 - accuracy: 0.1036
Epoch 5/240
120/120 [==============================] - 1s 9ms/step - loss: 2.3222 - accuracy: 0.1138
Epoch 6/240
120/120 [==============================] - 1s 8ms/step - loss: 2.2951 - accuracy: 0.1280
Epoch 7/240
120/120 [==============================] - 1s 8ms/step - loss: 2.2680 - accuracy: 0.1443
Epoch 8/240
120/120 [==============================] - 1s 9ms/step - loss: 2.2408 - accuracy: 0.1719
Epoch 9/240
120/120 [==============================] - 1s 9ms/step - loss: 2.2133 - accuracy: 0.2197
Epoch 10/240
120/120 [==============================] - 1s 9ms/step - loss: 2.1853 - accur

In [ ]:
stats(model)

loss= 0.31196504831314087
accuracy= 0.9154000282287598


# Teste 7x7


In [ ]:
teste7x7 = Input(shape=(inpx))
layer1 = Conv2D(32, kernel_size=(7, 7), activation='relu')(teste7x7)
flattened = Flatten()(layer1)  # Add this line to flatten the output of the Conv2D layer
layer2 = Dense(250, activation='sigmoid')(flattened)
layer3 = Dense(10, activation='softmax')(layer2)

In [ ]:
model7x7 = Model([teste7x7], layer3)
model7x7.compile(optimizer=keras.optimizers.Adadelta(),
			loss=keras.losses.categorical_crossentropy,
			metrics=['accuracy'])

model7x7.fit(x_train, y_train, epochs=240, batch_size=500)

Epoch 1/240
120/120 [==============================] - 2s 9ms/step - loss: 2.4876 - accuracy: 0.0975
Epoch 2/240
120/120 [==============================] - 1s 8ms/step - loss: 2.4448 - accuracy: 0.0975
Epoch 3/240
120/120 [==============================] - 1s 8ms/step - loss: 2.4035 - accuracy: 0.0988
Epoch 4/240
120/120 [==============================] - 1s 8ms/step - loss: 2.3629 - accuracy: 0.1054
Epoch 5/240
120/120 [==============================] - 1s 8ms/step - loss: 2.3229 - accuracy: 0.1195
Epoch 6/240
120/120 [==============================] - 1s 8ms/step - loss: 2.2837 - accuracy: 0.1368
Epoch 7/240
120/120 [==============================] - 1s 8ms/step - loss: 2.2458 - accuracy: 0.1529
Epoch 8/240
120/120 [==============================] - 1s 8ms/step - loss: 2.2093 - accuracy: 0.1798
Epoch 9/240
120/120 [==============================] - 1s 9ms/step - loss: 2.1742 - accuracy: 0.2485
Epoch 10/240
120/120 [==============================] - 1s 8ms/step - loss: 2.1400 - accura

In [ ]:
stats(model7x7)

loss= 0.31100741028785706
accuracy= 0.9160000085830688


#Teste 25x25


In [ ]:
teste25x25 = Input(shape=(inpx))
layer1 = Conv2D(32, kernel_size=(7, 7), activation='relu')(teste25x25)
flattened = Flatten()(layer1)  # Add this line to flatten the output of the Conv2D layer
layer2 = Dense(250, activation='sigmoid')(flattened)
layer3 = Dense(10, activation='softmax')(layer2)

In [ ]:
model25x25 = Model([teste25x25], layer3)
model25x25.compile(optimizer=keras.optimizers.Adadelta(),
			loss=keras.losses.categorical_crossentropy,
			metrics=['accuracy'])

model25x25.fit(x_train, y_train, epochs=240, batch_size=500)

Epoch 1/240
120/120 [==============================] - 2s 8ms/step - loss: 2.3554 - accuracy: 0.1022
Epoch 2/240
120/120 [==============================] - 1s 8ms/step - loss: 2.3330 - accuracy: 0.1023
Epoch 3/240
120/120 [==============================] - 1s 8ms/step - loss: 2.3110 - accuracy: 0.1029
Epoch 4/240
120/120 [==============================] - 1s 8ms/step - loss: 2.2890 - accuracy: 0.1071
Epoch 5/240
120/120 [==============================] - 1s 8ms/step - loss: 2.2665 - accuracy: 0.1165
Epoch 6/240
120/120 [==============================] - 1s 8ms/step - loss: 2.2432 - accuracy: 0.1331
Epoch 7/240
120/120 [==============================] - 1s 8ms/step - loss: 2.2190 - accuracy: 0.1779
Epoch 8/240
120/120 [==============================] - 1s 8ms/step - loss: 2.1936 - accuracy: 0.2792
Epoch 9/240
120/120 [==============================] - 1s 8ms/step - loss: 2.1667 - accuracy: 0.4123
Epoch 10/240
120/120 [==============================] - 1s 8ms/step - loss: 2.1383 - accura

In [ ]:
stats(model25x25)

loss= 0.3098340928554535
accuracy= 0.916700005531311


## Resultados parciais foram inconclusivos com apenas uma camada de convolução

- 3x3:

#

In [ ]:
inpx = Input(shape=inpx)
layer1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(inpx)
layer2 = Conv2D(64, (3, 3), activation='relu')(layer1)
layer3 = MaxPooling2D(pool_size=(3, 3))(layer2)
layer4 = Dropout(0.5)(layer3)
layer5 = Flatten()(layer4)
layer6 = Dense(250, activation='sigmoid')(layer5)
layer7 = Dense(10, activation='softmax')(layer6)

In [ ]:
model = Model([inpx], layer7)
model.compile(optimizer=keras.optimizers.Adadelta(),
			loss=keras.losses.categorical_crossentropy,
			metrics=['accuracy'])

model.fit(x_train, y_train, epochs=120, batch_size=500)


Epoch 1/120
120/120 [==============================] - 4s 23ms/step - loss: 2.2919 - accuracy: 0.1497
Epoch 2/120
120/120 [==============================] - 3s 23ms/step - loss: 2.2853 - accuracy: 0.1612
Epoch 3/120
120/120 [==============================] - 3s 27ms/step - loss: 2.2793 - accuracy: 0.1734
Epoch 4/120
120/120 [==============================] - 4s 37ms/step - loss: 2.2736 - accuracy: 0.1892
Epoch 5/120
120/120 [==============================] - 4s 37ms/step - loss: 2.2672 - accuracy: 0.2094
Epoch 6/120
120/120 [==============================] - 3s 25ms/step - loss: 2.2618 - accuracy: 0.2250
Epoch 7/120
120/120 [==============================] - 3s 23ms/step - loss: 2.2555 - accuracy: 0.2465
Epoch 8/120
120/120 [==============================] - 3s 23ms/step - loss: 2.2501 - accuracy: 0.2639
Epoch 9/120
120/120 [==============================] - 3s 23ms/step - loss: 2.2440 - accuracy: 0.2841
Epoch 10/120
120/120 [==============================] - 3s 23ms/step - loss: 2.237

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])


loss= 0.4918977916240692
accuracy= 0.8651000261306763
